In [15]:
import lightgbm as lgb
import pandas as pd # 基本ライブラリ
import numpy as np # 基本ライブラリ
import matplotlib.pyplot as plt # グラフ描画用

import data_get
import race_before_data
import time
import datetime
import sched

run_time = [
    '2022-04-03 11:49:00','2022-04-03 11:49:05','2022-04-03 11:49:10',
]

def func():
    print(time.time())

s = sched.scheduler(time.time)
for rt in run_time:
    t = time.strptime(rt, '%Y-%m-%d %H:%M:%S')
    t = time.mktime(t)
    s.enterabs(t, 1, func)
s.run()

1648955398.082921
1648955398.086427
1648955398.086616


In [19]:
date = '2022-04-03'
data_get.download_file('racelists', date)
df_race = data_get.get_racelists(date)

df_race.to_csv(f"dataset/rawdata_syussouhyou/{date}_race_list.csv", encoding='utf_8_sig')
df_race = df_race.sort_values(by="deadline")

data_get.download_file('results', date)
df_result = data_get.get_results(date)

deadlines_list = df_race["deadline"].values

In [20]:

deadlines_list

array(['08:59', '09:28', '09:57', '10:26', '10:30', '10:53', '10:54',
       '10:57', '11:05', '11:11', '11:19', '11:23', '11:24', '11:31',
       '11:34', '11:42', '11:46', '11:52', '11:52', '11:55', '11:58',
       '12:00', '12:14', '12:19', '12:21', '12:22', '12:25', '12:26',
       '12:44', '12:46', '12:50', '12:53', '12:53', '12:54', '13:10',
       '13:14', '13:14', '13:19', '13:21', '13:23', '13:26', '13:40',
       '13:45', '13:48', '13:48', '13:51', '13:53', '14:01', '14:10',
       '14:16', '14:18', '14:21', '14:24', '14:24', '14:40', '14:48',
       '14:48', '14:55', '14:58', '14:58', '15:15', '15:17', '15:19',
       '15:21', '15:26', '15:30', '15:33', '15:36', '15:39', '15:44',
       '15:49', '15:52', '15:55', '15:58', '15:58', '16:06', '16:09',
       '16:14', '16:15', '16:25', '16:27', '16:30', '16:30', '16:33',
       '16:37', '16:41', '16:48', '16:53', '17:00', '17:03', '17:05',
       '17:11', '17:21', '17:29', '17:38', '17:40', '17:47', '17:55',
       '18:05', '18:

In [21]:
format_deadlines = [date + " " + s + ":00" for s in deadlines_list]

In [32]:
def create_ex_time(x):
    date_format = '%Y-%m-%d %H:%M:%S'
    diff = datetime.timedelta(seconds=30)

    date_dt = datetime.datetime.strptime(x, date_format)
    date_dt_new = date_dt - diff

    date_str_new = date_dt_new.strftime(date_format)
    return date_str_new

In [33]:


list(map(create_ex_time, format_deadlines))

['2022-04-03 08:58:30',
 '2022-04-03 09:27:30',
 '2022-04-03 09:56:30',
 '2022-04-03 10:25:30',
 '2022-04-03 10:29:30',
 '2022-04-03 10:52:30',
 '2022-04-03 10:53:30',
 '2022-04-03 10:56:30',
 '2022-04-03 11:04:30',
 '2022-04-03 11:10:30',
 '2022-04-03 11:18:30',
 '2022-04-03 11:22:30',
 '2022-04-03 11:23:30',
 '2022-04-03 11:30:30',
 '2022-04-03 11:33:30',
 '2022-04-03 11:41:30',
 '2022-04-03 11:45:30',
 '2022-04-03 11:51:30',
 '2022-04-03 11:51:30',
 '2022-04-03 11:54:30',
 '2022-04-03 11:57:30',
 '2022-04-03 11:59:30',
 '2022-04-03 12:13:30',
 '2022-04-03 12:18:30',
 '2022-04-03 12:20:30',
 '2022-04-03 12:21:30',
 '2022-04-03 12:24:30',
 '2022-04-03 12:25:30',
 '2022-04-03 12:43:30',
 '2022-04-03 12:45:30',
 '2022-04-03 12:49:30',
 '2022-04-03 12:52:30',
 '2022-04-03 12:52:30',
 '2022-04-03 12:53:30',
 '2022-04-03 13:09:30',
 '2022-04-03 13:13:30',
 '2022-04-03 13:13:30',
 '2022-04-03 13:18:30',
 '2022-04-03 13:20:30',
 '2022-04-03 13:22:30',
 '2022-04-03 13:25:30',
 '2022-04-03 13:

In [34]:

run_time = [
    '2022-04-03 12:28:00','2022-04-03 11:49:05','2022-04-03 11:49:10',
]

def func():
    print(time.time())

s = sched.scheduler(time.time)
t = time.strptime(run_time[0], '%Y-%m-%d %H:%M:%S')
t = time.mktime(t)
s.enterabs(t, 1, func)
s.run()

1648956480.0027509


In [45]:
import data_get
import race_before_data
import pickle
import time
import datetime
import before_odds
import sched




def hensachi(x):
    x = x.astype(float)
    if x.std() == 0:
        return pd.Series(np.full(6, 50.0))

    hensa = (x - x.mean()) * 10 / x.std() + 50

    return hensa


def rank_mapper(x):
    if 'A1' in x:
        y = 4
    elif 'A2' in x:
        y = 3
    elif 'B1' in x:
        y = 2
    else:
        y = 1
    return y

def create_ex_time(x):
    date_format = '%Y-%m-%d %H:%M:%S'
    diff = datetime.timedelta(seconds=30)

    date_dt = datetime.datetime.strptime(x, date_format)
    date_dt_new = date_dt - diff

    date_str_new = date_dt_new.strftime(date_format)
    return date_str_new

def ex(index, row):
    time.sleep(1)
    place = row["place"]
    print(f"開催場所：{place}")
    place_cd, race_no, date = row["place_cd"], row["race_no"], row["date"]
    deadline = row["deadline"]
    print(f"レース番号：{race_no}")
    bi = race_before_data.get_beforeinfo(date, place_cd, race_no)
    tmp = pd.merge(pd.DataFrame(row).T, pd.DataFrame(bi).T, on=["race_no", "place_cd"])
    usecols = [f'{k}_{i}' for k in ('class', 'glob_win', 'glob_in2',
                                    'loc_win', 'loc_in2', 'moter_in2', 'boat_in2', 'EST', 'ESC')
               for i in range(1, 7)]
    usecols += [f'ET_{i}'
                for i in range(1, 7)]

    tmp = tmp[usecols]

    for i in range(1, 7):
        col_name_class = f"class_{i}"
        tmp[col_name_class] = tmp[col_name_class].map(rank_mapper)
    class_col = [s for s in tmp.columns if "class" in s]

    tmp.loc[:, class_col] = tmp.loc[:, class_col].apply(hensachi, axis=1)

    hensa_col = ['glob_win', 'glob_in2', 'loc_win', 'loc_in2', 'moter_in2', 'boat_in2', 'EST', "ET", "ESC"]

    for i in hensa_col:
        class_col = [s for s in tmp.columns if i in s]
        tmp[class_col] = tmp.loc[:, class_col].apply(hensachi, axis=1)

    X = tmp.values
    y_pred = model.predict(X)
    # テストデータのクラス予測確率 (各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率,クラス2の予測確率...] を返す)
    y_pred_prob = model.predict_proba(X)
    hoge = before_odds.get_odds(BET_TYPE="tf", race_no=race_no, place_cd=place_cd, date=date)

    before_odds_list = []

    for i in hoge.select("td.oddsPoint"):
        before_odds_list.append(i.text)

    hoge = [float(s) for s in before_odds_list]
    # print(f"予測確率：{y_pred_prob}")
    # print(f"直前オッズ:{hoge}")
    # print(f"オッズ＊予測{hoge*y_pred_prob}")
    oddsinpred = np.argmax(hoge * y_pred_prob)
    print(np.max(hoge * y_pred_prob))
    print(f"開催場所：{place}, レース番号：{race_no},オッズ込み予想{oddsinpred + 1},オッズ{hoge[oddsinpred]}")


date = '2022-04-03'
data_get.download_file('racelists', date)
df_race = data_get.get_racelists(date)

df_race.to_csv(f"dataset/rawdata_syussouhyou/{date}_race_list.csv", encoding='utf_8_sig')
df_race = df_race.sort_values(by="deadline").reset_index(drop=True)
deadlines_list = df_race["deadline"].values

filename = '0402model.sav'
model = pickle.load(open(filename, 'rb'))

format_deadlines = [date + " " + s + ":00" for s in deadlines_list]
ex_time_list = list(map(create_ex_time, format_deadlines))





for index, row in df_race.iterrows():

    ex_time = ex_time_list[index]

    try:
        s = sched.scheduler(time.time)
        t = time.strptime(ex_time, '%Y-%m-%d %H:%M:%S')
        t = time.mktime(t)
        s.enterabs(t, 1, ex,kwargs={'index':index,'row':row})
        s.run()
        print("============================")


    except Exception as e:
        print("hoge")
        print(e, flush=True)

2022-04-03 08:58:30
0
開催場所：芦屋
レース番号：1
締切時刻：08:59

オッズ込み予想5,オッズ15.8
2022-04-03 09:27:30
1
開催場所：芦屋
レース番号：2
締切時刻：09:28

オッズ込み予想1,オッズ1.2
2022-04-03 09:56:30
2
開催場所：芦屋
レース番号：3
締切時刻：09:57

オッズ込み予想1,オッズ1.3
2022-04-03 10:25:30
3
開催場所：芦屋
レース番号：4
締切時刻：10:26

オッズ込み予想4,オッズ6.0
2022-04-03 10:29:30
4
開催場所：津
レース番号：1
締切時刻：10:30

オッズ込み予想1,オッズ1.0
2022-04-03 10:52:30
5
開催場所：戸田
レース番号：1
締切時刻：10:53

オッズ込み予想1,オッズ4.2
2022-04-03 10:53:30
6
開催場所：芦屋
レース番号：5
締切時刻：10:54

オッズ込み予想3,オッズ150.3
2022-04-03 10:56:30
7
開催場所：津
レース番号：2
締切時刻：10:57

オッズ込み予想4,オッズ6.9
2022-04-03 11:04:30
8
開催場所：常滑
レース番号：1
締切時刻：11:05

オッズ込み予想6,オッズ22.1
2022-04-03 11:10:30
9
開催場所：児島
レース番号：2
締切時刻：11:11

オッズ込み予想3,オッズ15.2
2022-04-03 11:18:30
10
開催場所：戸田
レース番号：2
締切時刻：11:19

オッズ込み予想1,オッズ3.3
2022-04-03 11:22:30
11
開催場所：芦屋
レース番号：6
締切時刻：11:23

オッズ込み予想4,オッズ5.8
2022-04-03 11:23:30
12
開催場所：津
レース番号：3
締切時刻：11:24

オッズ込み予想6,オッズ20.4
2022-04-03 11:30:30
13
開催場所：常滑
レース番号：2
締切時刻：11:31

オッズ込み予想2,オッズ33.0
2022-04-03 11:33:30
14
開催場所：浜名湖
レース番号：1
締切時刻：11:34

オッズ込み予想2,オッズ3.3


KeyboardInterrupt: 

In [44]:
df_race["deadline"]

0      08:59
1      09:28
2      09:57
3      10:26
43     10:30
       ...  
113    20:05
41     20:14
114    20:37
23     20:39
42     20:43
Name: deadline, Length: 115, dtype: object